In [116]:
import numpy as np
import math
import scipy
from scipy import spatial
from scipy.spatial import ConvexHull
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go

In [117]:
def different_regions(cube):
    unique, counts = np.unique(cube, return_counts=True)
    return dict(zip(unique, counts))

def get_region_coordinates(matrix):
    xx = matrix.shape[0]
    yy = matrix.shape[1]
    zz = matrix.shape[2]
    
    u = np.unique(matrix).tolist()
    
    regions = []
    for i in range(len(u)):
        regions.append(np.zeros([3, 0]))
    
    for x in range(xx):
        for y in range(yy):
            for z in range(zz):
                idx = u.index(matrix[x,y,z])
                regions[idx]=np.append(regions[idx],np.array([[x],[y],[z]]),axis=1)
    return u, regions

def convex_hull_mesh(regions):
    for i in range(len(regions)):
        hull = scipy.spatial.ConvexHull(regions[i].T)
        regions[i]=hull.points[np.unique(hull.simplices)].T
    return regions

def get_colors(n):
    ColourValues=["FF0000", "00FF00", "0000FF", "FFFF00", "FF00FF", "00FFFF", "000000",
        "800000", "008000", "000080", "808000", "800080", "008080", "808080",
        "C00000", "00C000", "0000C0", "C0C000", "C000C0", "00C0C0", "C0C0C0",
        "400000", "004000", "000040", "404000", "400040", "004040", "404040",
        "200000", "002000", "000020", "202000", "200020", "002020", "202020",
        "600000", "006000", "000060", "606000", "600060", "006060", "606060",
        "A00000", "00A000", "0000A0", "A0A000", "A000A0", "00A0A0", "A0A0A0",
        "E00000", "00E000", "0000E0", "E0E000", "E000E0", "00E0E0", "E0E0E0"]
    M=len(ColourValues)
    color_list=[]
    for i in range(n):
        color_list.append('#'+ColourValues[(i%M)])
    return color_list

def get_traces(regions):
    colors = get_colors(len(regions))
    traces = []

    for i in range(len(regions)):
        traces.append(go.Mesh3d(x=regions[i][0],y=regions[i][1],z=regions[i][2],opacity=0.50,alphahull=5,
                           color=colors[i]))
        
    return traces

# GENERATE DATA

## DATA TYPE 1

In [248]:
dim=20

m=dim
n=dim
p=dim

a=np.empty([m,n,p])

for x in range(m):
    for y in range(n):
        for z in range(p):
            if (((x-m/2)**2+(y-n/2)**2+(z-n/2)**2)<((m/4))**2):
                a[x][y][z]=10
            elif (((x-m/2)**2+(y-n/2)**2+(z-n/2)**2)<((m/2)-1)**2):
                a[x][y][z]=5
            else:
                a[x][y][z]=0

text_file = open("fig1_"+str(dim)+".txt", "w")

for x in range(m):
    for y in range(n):
        for z in range(p):
            text_file.write(str(int(a[x,y,z]))+'\n')
text_file.close()

## DATA TYPE 2

In [292]:
#CUBE 3

dim=20

m=dim
n=dim
p=dim

a=np.empty([m,n,p])

for x in range(m):
    for y in range(n):
        for z in range(p):
            a[x][y][z] = (math.floor(z/((p+1)/3)))
            if (y in range(math.floor(11*n/20), math.floor(18*n/20))):
                a[x][y][z]+=100
            elif (y>(3*n/4)):
                a[x][y][z]+=200
            if (x>(4*p/20)):
                a[x][y][z]+=400
                
text_file = open("fig2_"+str(dim)+".txt", "w")

for x in range(m):
    for y in range(n):
        for z in range(p):
            text_file.write(str(int(a[x,y,z]))+'\n')
text_file.close()

# PLOT GENERATED DATA

In [291]:
#Check different regions
different_regions(a)

{0.0: 523908,
 1.0: 517440,
 2.0: 510972,
 100.0: 333396,
 101.0: 329280,
 102.0: 325164,
 200.0: 95256,
 201.0: 94080,
 202.0: 92904,
 400.0: 2042172,
 401.0: 2016960,
 402.0: 1991748,
 500.0: 1299564,
 501.0: 1283520,
 502.0: 1267476,
 600.0: 371304,
 601.0: 366720,
 602.0: 362136}

In [280]:
values, regions= get_region_coordinates(a)
traces= get_traces(regions)

In [285]:
# 3D plot layout settings
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        nticks=2, range = [0,20],),
                    yaxis = dict(
                        nticks=2, range = [0,20],),
                    zaxis = dict(
                        nticks=2, range = [0,20],),),
                  )

# Plot figure
plotly.offline.init_notebook_mode(connected=True)
fig = go.Figure(data=traces, layout=layout)

plotly.offline.iplot(fig)

# CHECK RESULT

In [106]:
with open("Result_final.txt") as f:
    data = f.readlines()

In [107]:
b=np.empty([m,n,p])

i=0
for x in range(m):
    for y in range(n):
        for z in range(p):
            b[x][y][z] = int(data[i])
            i+=1

In [108]:
different_regions(b)

{1.0: 5031, 2.0: 2969}

In [110]:
values, regions= get_region_coordinates(b)
traces = get_traces(regions)

plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(traces)